### Build stock analysis agents with browsing, sec search and news search
First, uncomment below to install necessary packages

In [1]:
# %pip install crewai
# %pip install langchain
# %pip install unstructured
# %pip install sec_api
# %pip install alpha_vantage
# %pip install --upgrade --quiet  google-search-results
#% pip install --upgrade --quiet  yfinance

From yesterday's RAG introduction, it is natural to further make the process to be autonomous. For example, previously, we downloaded the files using the sec_edgar api, but in today's session, we will be using sec_api and grant it as a the 'Tool' for our agent, this is equivalent to 'actions' from GPTs when we try to create our custom GPTs. 

As mentioned, we want to grant the model tools and automate the observe, reasoning and action process.


There are a lot of pre-built toolings from LangChain: https://python.langchain.com/docs/integrations/tools/. For finance related toolings, we have alpha_vantage, google finance and yahoo finance news integrated, as shown below.

LangChain's agents chain follows similarly to the ReAct framework: 
Agent will follow the chain of:
- thought
- observation
- action
"https://arxiv.org/pdf/2210.03629.pdf"

In [26]:
import os
from langchain.agents import AgentType, initialize_agent
import langchain_community.utilities.alpha_vantage as av
from langchain_openai import ChatOpenAI

alpha_vantage = av.AlphaVantageAPIWrapper(alphavantage_api_key=os.environ["ALPHAVANTAGE_API_KEY"])
alpha_vantage.run("USD", "JPY")

{'1. From_Currency Code': 'USD',
 '2. From_Currency Name': 'United States Dollar',
 '3. To_Currency Code': 'JPY',
 '4. To_Currency Name': 'Japanese Yen',
 '5. Exchange Rate': '148.19100000',
 '6. Last Refreshed': '2024-01-19 16:22:01',
 '7. Time Zone': 'UTC',
 '8. Bid Price': '148.18720000',
 '9. Ask Price': '148.19770000'}

In [27]:
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper
from langchain_community.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper()
tool = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper(serp_api_key=os.environ["SERP_API_KEY"]))
tool.run("Google")

'\nQuery: Google\nstock: GOOGL:NASDAQ\nprice: $145.08\npercentage: 1.12\nmovement: Up\nus: price = 37532.41, movement = Up\neurope: price = 16539.66, movement = Down\nasia: price = 35963.27, movement = Up\n'

An illustration of ReAct framework:

In [28]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
llm = ChatOpenAI(temperature=0.1, model='gpt-4-1106-preview')
tools = [YahooFinanceNewsTool()]
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
agent_chain.run(
    "Any news about Apple today?",
)



> Entering new AgentExecutor chain...
I need to check the latest financial news about Apple Inc.
Action: yahoo_finance_news
Action Input: AAPL
Observation: Is Trending Stock Apple Inc. (AAPL) a Buy Now?
Apple (AAPL) has been one of the stocks most watched by Zacks.com users lately. So, it is worth exploring what lies ahead for the stock.
Thought:I now know the final answer
Final Answer: There is news about Apple today, with a headline discussing whether Apple Inc. (AAPL) is a good stock to buy now, indicating it has been receiving attention from users on Zacks.com.

> Finished chain.


'There is news about Apple today, with a headline discussing whether Apple Inc. (AAPL) is a good stock to buy now, indicating it has been receiving attention from users on Zacks.com.'

For the project, we will enable the agent to:
- Search: search google and google results to obtain the relevant urls
- Scrape: browse the link through web scraping
- Analyze fillings: analyze the 10-Q and 10-K documents through interacting with the sec_api
- Calculate: use calculator tools to calculate some simple metrics like PE ratios etc.

It is important to understand from a higher level, that the language model will:
- pick up the tools from conversation after thought process
- take actions by using the tool, notice the @tool decorator
- observe the result and reflect

Below are implementations of tools mentioned above similarly to pre-built integrations where agent will use the tools, here is an example of tool that scrape a url and summarized the content, with browserless api:

In [40]:
import json
import os
import requests
from crewai import Agent, Task
from langchain.tools import tool
from unstructured.partition.html import partition_html
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview")

# an example of a tool that can be used to scrape and summarize a url content
class BrowserTools:
    @tool("Scrape website content")
    def scrape_and_summarize_website(website):
        """Useful to scrape and summarize a website content"""
        url = f"https://chrome.browserless.io/content?token={os.environ['BROWSERLESS_API_KEY']}"
        payload = json.dumps({"url": website})
        headers = {"cache-control": "no-cache", "content-type": "application/json"}
        response = requests.request("POST", url, headers=headers, data=payload)
        elements = partition_html(text=response.text)
        content = "\n\n".join([str(el) for el in elements])
        content = [content[i : i + 8000] for i in range(0, len(content), 8000)]
        summaries = []
        for chunk in content:
            agent = Agent(
                role="Principal Researcher",
                goal="Do comprehensive research and summaries based on the content you are working with",
                backstory="You're a Principal Researcher at a Hedge Fund and you need to do research about the context you are given.",
                allow_delegation=False,
                llm=llm,
            )
            task = Task(
                agent=agent,
                description=f"Analyze and summarize the content below, make sure to include the most relevant information in the summary, return only the summary nothing else.\n\nCONTENT\n----------\n{chunk}",
            )
            summary = task.execute()
            summaries.append(summary)
        return "\n\n".join(summaries)

Below is a tool that will enable agent to make calculations.

In [41]:
from langchain.tools import tool

class CalculatorTools:
    @tool("Make a calcualtion")
    def calculate(operation):
        """Useful to perform any mathematica calculations,
        like sum, minus, mutiplcation, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        return eval(operation)

Below is a tool that will enable agent to search sec fillings automatically with sec_api, and return the most relevant chunks from the sec fillings, exactly how we built RAG yesterday, using cohere's embedding and FAISS vector store

In [42]:
import os
import requests
from langchain.tools import tool
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import FAISS
from sec_api import QueryApi
from unstructured.partition.html import partition_html


class SECTools:
    @tool("Search 10-Q form")
    def search_10q(data):
        """
        Useful to search information from the latest 10-Q form for a
        given stock.
        The input to this tool should be a pipe (|) separated text of
        length two, representing the stock ticker you are interested, what
        question you have from it.
        For example, `AAPL|what was last quarter's revenue`.
        """
        stock, ask = data.split("|")
        queryApi = QueryApi(api_key=os.environ["SEC_API_API_KEY"])
        query = {
            "query": {"query_string": {"query": f'ticker:{stock} AND formType:"10-Q"'}},
            "from": "0",
            "size": "1",
            "sort": [{"filedAt": {"order": "desc"}}],
        }

        filings = queryApi.get_filings(query)["filings"]
        link = filings[0]["linkToFilingDetails"]
        answer = SECTools.__embedding_search(link, ask)
        return answer

    @tool("Search 10-K form")
    def search_10k(data):
        """
        Useful to search information from the latest 10-K form for a
        given stock.
        The input to this tool should be a pipe (|) separated text of
        length two, representing the stock ticker you are interested, what
        question you have from it.
        For example, `AAPL|what was last year's revenue`.
        """
        stock, ask = data.split("|")
        queryApi = QueryApi(api_key=os.environ["SEC_API_API_KEY"])
        query = {
            "query": {"query_string": {"query": f'ticker:{stock} AND formType:"10-K"'}},
            "from": "0",
            "size": "1",
            "sort": [{"filedAt": {"order": "desc"}}],
        }

        filings = queryApi.get_filings(query)["filings"]
        link = filings[0]["linkToFilingDetails"]
        answer = SECTools.__embedding_search(link, ask)
        return answer

# some stocks don't have 10-k forms but instead have 8-k forms
    # @tool("Search 8-K form")
    # def search_8k(data):
    #     """
    #     Useful to search information from the latest 8-K form for a
    #     given stock.
    #     The input to this tool should be a pipe (|) separated text of
    #     length two, representing the stock ticker you are interested, what
    #     question you have from it.
    #     For example, `AAPL|what was last year's revenue`.
    #     """
    #     stock, ask = data.split("|")
    #     queryApi = QueryApi(api_key=os.environ["SEC_API_API_KEY"])
    #     query = {
    #         "query": {"query_string": {"query": f'ticker:{stock} AND formType:"8-K"'}},
    #         "from": "0",
    #         "size": "1",
    #         "sort": [{"filedAt": {"order": "desc"}}],
    #     }

    #     filings = queryApi.get_filings(query)["filings"]
    #     link = filings[0]["linkToFilingDetails"]
    #     answer = SECTools.__embedding_search(link, ask)
    #     return answer

    def __embedding_search(url, ask):
        text = SECTools.__download_form_html(url)
        elements = partition_html(text=text)
        content = "\n".join([str(el) for el in elements])
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=2000,
            chunk_overlap=150,
            length_function=len,
            is_separator_regex=False,
        )
        docs = text_splitter.create_documents([content])
        retriever = FAISS.from_documents(docs, CohereEmbeddings(cohere_api_key=os.environ["COHERE_API_KEY"])).as_retriever()
        answers = retriever.get_relevant_documents(ask, top_k=4)
        answers = "\n\n".join([a.page_content for a in answers])
        return answers

    def __download_form_html(url):
        headers = {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7",
            "Cache-Control": "max-age=0",
            "Dnt": "1",
            "Sec-Ch-Ua": '"Not_A Brand";v="8", "Chromium";v="120"',
            "Sec-Ch-Ua-Mobile": "?0",
            "Sec-Ch-Ua-Platform": '"macOS"',
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Upgrade-Insecure-Requests": "1",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        }

        response = requests.get(url, headers=headers)
        return response.text

Below is a tool that enable agent to search the news and internet and return the relevant links with serper api.

In [43]:
import json
import os
import requests
from langchain.tools import tool


class SearchTools:
    @tool("Search the internet")
    def search_internet(query):
        """Useful to search the internet
        about a a given topic and return relevant results"""
        top_result_to_return = 3
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q": query})
        headers = {
            "X-API-KEY": os.environ["SERPER_API_KEY"],
            "content-type": "application/json",
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        results = response.json()["organic"]
        string = []
        for result in results[:top_result_to_return]:
            try:
                string.append(
                    "\n".join(
                        [
                            f"Title: {result['title']}",
                            f"Link: {result['link']}",
                            f"Snippet: {result['snippet']}",
                            "\n-----------------",
                        ]
                    )
                )
            except KeyError:
                next

        return "\n".join(string)

    @tool("Search news on the internet")
    def search_news(query):
        """Useful to search news about a company, stock or any other
        topic and return relevant results""" ""
        top_result_to_return = 3
        url = "https://google.serper.dev/news"
        payload = json.dumps({"q": query})
        headers = {
            "X-API-KEY": os.environ["SERPER_API_KEY"],
            "content-type": "application/json",
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        results = response.json()["news"]
        string = []
        for result in results[:top_result_to_return]:
            try:
                string.append(
                    "\n".join(
                        [
                            f"Title: {result['title']}",
                            f"Link: {result['link']}",
                            f"Snippet: {result['snippet']}",
                            "\n-----------------",
                        ]
                    )
                )
            except KeyError:
                next

        return "\n".join(string)

CrewAI enable to build swarm of agents, as agents have their respective role, goal, backstory, and tools assigned to them.


A high level of such design would be think about tasks for the final goal, and create agents that are suitable for certain tasks.

In the below exapmle, we will create 3 agents:
- financial analyst that conduct traditional stock analysis
- research analyst that analyze news, market trend, and sentiment
- investment advisor that combine previous analysts' results to make an inventment decision

In [44]:
from crewai import Agent
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model="gpt-4-1106-preview")

class StockAnalysisAgents:
    def financial_analyst(self):
        return Agent(
            role="The Best Financial Analyst",
            goal="""Impress all customers with your financial data and market trends analysis""",
            backstory="""The most seasoned financial analyst with lots of expertise in stock market analysis and investment strategies that is working for a super important customer.""",
            verbose=True,
            llm=llm,
            tools=[
                BrowserTools.scrape_and_summarize_website,
                SearchTools.search_internet,
                CalculatorTools.calculate,
                SECTools.search_10q,
                SECTools.search_10k,
            ],
        )

    def research_analyst(self):
        return Agent(
            role="Staff Research Analyst",
            goal="""Being the best at gather, interpret data and amaze your customer with it""",
            backstory="""Known as the best research analyst, you're skilled in sifting through news, company announcements,
            and market sentiments. Now you're working on a super important customer""",
            verbose=True,
            llm=llm,
            tools=[
                BrowserTools.scrape_and_summarize_website,
                SearchTools.search_internet,
                SearchTools.search_news,
            ],
        )

    def investment_advisor(self):
        return Agent(
            role="Private Investment Advisor",
            goal="""Impress your customers with full analyses over stocks and completer investment recommendations""",
            backstory="""You're the most experienced investment advisor and you combine various analytical insights to formulate strategic investment advice. You are now working for a super important customer you need to impress.""",
            verbose=True,
            llm=llm,
            tools=[
                BrowserTools.scrape_and_summarize_website,
                SearchTools.search_internet,
                SearchTools.search_news,
                CalculatorTools.calculate,
            ],
        )


/opt/homebrew/Caskroom/miniforge/base/envs/capstone/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


We will create different tasks for different agents, in below example, we have four tasks:
- research of a company based on news, press releases and market analysis: this task can be suitable for our research analyst
- financial and filling analysis to gather key metrics like PE, EPS, SEC fillings: this task is suitable for financial analyst
- recommend task: for our investment advisor

In [45]:
from crewai import Task
from textwrap import dedent


class StockAnalysisTasks:
    def research(self, agent, company):
        return Task(
            description=dedent(f"""
                Conduct a focused research on {company}, analyzing recent news, press releases, and market analyses.
                Examine events like mergers, acquisitions, and regulatory changes affecting stock performance.
                Assess market sentiments and forecasts, and identify key future events like earnings reports.
                Include sentiment analysis and the stock ticker for {company}.
                Ensure accuracy and relevance in a well-structured report offering actionable insights.
                Remember, the quality of your research is critical for strategic decisions. Company: {company}
                """
            ),
            agent=agent,
        )

    def financial_analysis(self, agent):
        return Task(
            description=dedent(f"""
            Perform a detailed financial analysis focusing on key metrics like P/E ratio, EPS growth, revenue trends, and debt-to-equity ratio.
            Compare with industry peers and market indices, identifying financial strengths or weaknesses.
            Interpret these indicators to provide strategic insights and forward-looking statements.
            Ensure recent, relevant data supports your comprehensive financial assessment.
            """
            ),
            agent=agent,
        )

    def filings_analysis(self, agent):
        return Task(
            description=dedent(f"""
            Analyze the latest 10-Q and 10-K filings from EDGAR, focusing on Management's Discussion, financial statements, insider trading, and disclosed risks.
            Extract and highlight critical data, trends, and potential impacts on the stock's future performance.
            Your report should clarify the company's financial position and strategy, aiding informed investment decisions.
            """
            ),
            agent=agent,
        )

    def recommend(self, agent):
        return Task(
            description=dedent(f"""
            Integrate insights from financial and research analysis to formulate a strategic investment recommendation.
            Evaluate the company's financial health, market sentiment, qualitative insights from filings, and impact of significant events.
            Provide a reasoned argument for or against investment, in a clear, engaging report.
            Your nuanced, evidence-backed recommendation should guide informed investment decisions.
            """
            ),
            agent=agent,
        )


Finally, assemble the team and run

In [46]:
from crewai import Crew
from textwrap import dedent


class FinancialCrew:
    def __init__(self, company):
        self.company = company

    def run(self):
        agents = StockAnalysisAgents()
        tasks = StockAnalysisTasks()

        research_analyst_agent = agents.research_analyst()
        financial_analyst_agent = agents.financial_analyst()
        investment_advisor_agent = agents.investment_advisor()

        research_task = tasks.research(research_analyst_agent, self.company)
        financial_task = tasks.financial_analysis(financial_analyst_agent)
        filings_task = tasks.filings_analysis(financial_analyst_agent)
        recommend_task = tasks.recommend(investment_advisor_agent)

        crew = Crew(
            agents=[
                research_analyst_agent,
                financial_analyst_agent,
                investment_advisor_agent,
            ],
            tasks=[research_task, financial_task, filings_task, recommend_task],
            verbose=True,
        )

        result = crew.kickoff()
        return result

In [49]:
if __name__ == "__main__":
    print("## Welcome to Financial Analysis Crew")
    print("-------------------------------")
    company = input(
        dedent(
            """
      What is the company you want to analyze?
    """
        )
    )

    financial_crew = FinancialCrew(company)
    result = financial_crew.run()
    print("\n\n########################")
    print("## Here is the Report")
    print("########################\n")
    print(result)

## Welcome to Financial Analysis Crew
-------------------------------
Working Agent: Staff Research Analyst
Starting Task: 
Conduct a focused research on Nvidia, analyzing recent news, press releases, and market analyses.
Examine events like mergers, acquisitions, and regulatory changes affecting stock performance.
Assess market sentiments and forecasts, and identify key future events like earnings reports.
Include sentiment analysis and the stock ticker for Nvidia.
Ensure accuracy and relevance in a well-structured report offering actionable insights.
Remember, the quality of your research is critical for strategic decisions. Company: Nvidia
 ...


> Entering new CrewAgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search news on the internet
Action Input: Nvidia recent newsTitle: Magnificent Seven Stocks To Buy And Watch: Apple, Nvidia, Tesla Sell Off
Link: https://www.investors.com/research/magnificent-seven-stocks-to-buy-and-and-watch/
Snippet: Magnificent S

Created a chunk of size 2690, which is longer than the specified 2000
Created a chunk of size 2908, which is longer than the specified 2000
Created a chunk of size 2392, which is longer than the specified 2000
Created a chunk of size 2528, which is longer than the specified 2000
Created a chunk of size 2199, which is longer than the specified 2000
Created a chunk of size 2406, which is longer than the specified 2000
Created a chunk of size 2499, which is longer than the specified 2000
Created a chunk of size 2954, which is longer than the specified 2000


http://www.nvidia.com
, as soon as reasonably practicable after we electronically file such material with, or furnish it to, the Securities and Exchange Commission, or the SEC. The SEC’s website,
http://www.sec.gov
, contains reports, proxy and information statements, and other information regarding issuers that file electronically with the SEC. Our web site and the information on it or connected to it are not a part of this Annual Report on Form 10-K.
ITEM 1A. RISK FACTORS
In evaluating NVIDIA, the following risk factors should be considered in addition to the other information in this Annual Report on Form 10-K. Purchasing or owning NVIDIA common stock involves investment risks including, but not limited to, the risks described below. Any one of the following risks could harm our business, financial condition, results of operations or reputation, which could cause our stock price to decline, and you may lose all or a part of your investment. Additional risks, trends and uncertainties